<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین چهارم (طبقه‌بندی و خوشه‌بندی)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        دسترسی به داده‌ها و مدل‌های ذخیره شده
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        به دلیل حجم بالای فایل‌های ذخیره شده، از قرار دادن آن‌ها در پوشه نوتبوک صرف نظر کرده و با اجرای قطعه کد زیر، از طریق گوگل درایو به آن‌ها دسترسی پیدا می‌کنیم.
    </p>
</div>

In [ ]:
# import sys
# from google.colab import drive
# drive.mount('/content/drive')
# address = "University/Term 6/MIR/Homeworks/HW3"
# sys.path.append(f"/content/drive/My Drive/{address}")
# %cd /content/drive/My\ Drive/$address

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [ ]:
# !pip install bs4
# !pip install tqdm
# !pip install pandas
# !pip install requests
# !pip install hazm
# !pip install unidecode
# !pip install fasttext
# !pip install scikit-learn
# !pip install pandas
# !pip install nltk
# !pip install torch
# !pip install transformers
# !pip install sentence_transformers
# !pip install faiss-cpu
# !pip install faiss-gpu

In [1]:
import os
import re
import csv
import hazm
import nltk
import json
import torch
from tabulate import tabulate
import pickle
import zipfile
import requests
import fasttext
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from bs4 import BeautifulSoup
from string import punctuation
from IPython.display import display
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٦٨ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [2]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

CATEGORIES_CLASSES = {
    'Politics': 0,
    'World': 1,
    'Economy': 2,
    'Society': 3,
    'City': 4,
    'Sport': 5,
    'Science': 6,
    'Culture': 7,
    'IT': 8,
    'LifeSkills': 9,
}

CLASSES_CATEGORIES = {v: k for k, v in CATEGORIES_CLASSES.items()}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [3]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    os.remove('dataset.csv')
    return dataset


dataset = pd.DataFrame(read_dataset_from_file())

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [4]:
def display_dataset_info():
    global CATEGORIES, dataset

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for _, data in dataset.iterrows():
        length_dict[data['category']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,68362
1,سیاسی,15798
2,جهانی,2895
3,اقتصادی,8900
4,اجتماعی,13585
5,شهری,3853
6,ورزشی,8348
7,علمی,3190
8,فرهنگی,6512
9,فناوری اطلاعات,437


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. این مراحل عبارتند از:
    </p>
</div>

<div dir="auto" align="justify">
    <li style='direction:rtl;text-align:justify;'>
        نرمال سازی داده‌ها (normalize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف لینک‌ها (remove_links)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف نشانه‌های نگارشی (remove_punctuations)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        واحد سازی داده‌ها (word_tokenize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف کلمات نامعتبر (remove_invalid_words)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف ایست‌واژه‌ها (remove_stopwords)
    </li>
</div>

In [5]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

In [6]:
def save_preprocessed_texts(texts, path="Preprocessed_texts.pickle"):
    with open(path, "wb") as file:
        pickle.dump(texts, file)


def load_preprocessed_texts(path="Preprocessed_texts.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)


def data_to_text(data):
    return ' '.join([data['title']]).lower()


def get_preprocessed_texts(dataset, preprocessor, mode, save=False):
    preprocessed_texts = []
    if mode == 'process':
        texts = [data_to_text(data) for _, data in dataset.iterrows()]
        preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    if mode == 'load':
        preprocessed_texts = load_preprocessed_texts()
    if save:
        save_preprocessed_texts(preprocessed_texts)
    return preprocessed_texts

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Preprocessor ایجاد کرده و شروع به پیش پردازش داده‌ها می‌کنیم، یا داده‌های پیش‌پردازش‌شده‌ی ذخیره‌شده را از فایل مربوطه بازیابی می‌کنیم.
    </p>
</div>

In [7]:
preprocessor = Preprocessor(stopwords_path='stopwords.txt')
preprocessed_texts = get_preprocessed_texts(dataset, preprocessor, mode='load', save=False)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با توجه به این که حجم مجموعه داده‌ها باعث ایجاد محدودیت حافظه و زمان برای بعضی از مدل‌ها می‌شود، ناچار هستیم کسری از مجموعه‌ی داده‌ها را به عنوان ورودی برای یادگیری به آن‌ها بدهیم. با اجرای قطعه کد زیر، یک مجموعه داده‌ی کوچک‌تر از روی مجموعه داده‌ی اصلی ایجاد می‌کنیم.
    </p>
</div>

In [8]:
def get_mini_dataset(len_each_category=400):
    global CATEGORIES, dataset

    mini_dataset = []
    for category in CATEGORIES.keys():
        dataset_by_category = dataset.loc[dataset['category'] == category]
        length = min(len_each_category, dataset_by_category.shape[0])
        mini_dataset.append(dataset_by_category.sample(length, random_state=1))

    mini_dataset = pd.concat(mini_dataset).reset_index(drop=True)
    texts = [data_to_text(data) for _, data in mini_dataset.iterrows()]
    mini_preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    return mini_dataset, mini_preprocessed_texts


mini_dataset, mini_preprocessed_texts = get_mini_dataset(1000)

100%|██████████| 9437/9437 [00:02<00:00, 4148.33it/s]


In [9]:
mini_dataset['category_code'] = mini_dataset['category'].apply(lambda x: CATEGORIES_CLASSES[x])
labels = mini_dataset['category_code'].to_numpy()

In [10]:
vocabulary = {}

with open('vocabulary.pickle', "rb") as fp:
            vocabulary = pickle.load(fp)


In [11]:
class TF_IDF:

    def __init__(self, vocabulary):
        self.vectorizer = TfidfVectorizer(vocabulary=vocabulary)
        self.vectors = None

    def fit_transform_vectorizer(self, dataset):
        self.vectors = self.vectorizer.fit_transform(list(map(lambda doc: ' '.join(doc), dataset)))
        self.words = self.vectorizer.get_feature_names_out()
        dense_vectors = self.vectors.todense().tolist()
        return np.array(dense_vectors), self.words

    def save_TF_IDF_model(self, path="TF_IDF_model.pickle"):
        with open(path, "wb") as file:
            pickle.dump(self.vectors.todense().tolist(), file)

    def load_TF_IDF_model(self, path="TF_IDF_model.pickle"):
        with open(path, "rb") as file:
            return pickle.load(file)

In [12]:
TF_IDF_model = TF_IDF(vocabulary)
vectors, features = TF_IDF_model.fit_transform_vectorizer(mini_preprocessed_texts)

In [ ]:
train_vectors, test_vectors, train_labels, test_labels = train_test_split(vectors, labels,  test_size=0.15, shuffle=True)

In [14]:
clf = LogisticRegression(max_iter=1000, random_state=0)
clf.fit(train_vectors, train_labels)

LogisticRegression(max_iter=1000, random_state=0)

In [29]:
query = 'دعوای اردوغان و گلزار'
query = 'سرقت گوشی آیفون با نمایشگر اولد از کیف پوتین'


q_em = TF_IDF_model.vectorizer.transform([query])

p = clf.predict(q_em)
CLASSES_CATEGORIES[p[0]]
#p = clf.predict_proba(q_em)
#p

'IT'

In [77]:
mean_acc = clf.score(test_vectors, test_labels)
y = clf.predict(test_vectors)
f1_macro = f1_score(test_labels, y, labels=np.arange(10), average='macro')
conf_matrix = confusion_matrix(test_labels, y, labels=np.arange(10))
print(f"Mean Accurracy: {round(mean_acc, 3)}")
print(f"F1_Macro: {round(f1_macro, 3)}")
c_m_df = pd.DataFrame(conf_matrix).rename(columns=CLASSES_CATEGORIES, index=CLASSES_CATEGORIES)
c_m_df

Mean Accurracy: 0.732
F1_Macro: 0.72


,Politics,World,Economy,Society,City,Sport,Science,Culture,IT,LifeSkills
Politics,91,18,3,4,7,2,4,7,0,2
World,14,105,1,8,1,8,12,10,2,2
Economy,12,6,118,15,7,0,2,1,2,0
Society,4,1,5,100,15,0,10,16,1,5
City,6,1,7,6,112,1,5,9,0,3
Sport,4,0,0,4,1,147,0,1,0,1
Science,0,4,0,4,0,0,95,9,4,18
Culture,5,2,1,2,1,1,6,116,0,3
IT,3,6,4,2,2,1,14,5,33,1
LifeSkills,0,2,0,3,0,2,10,8,1,119
